In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:
HOME=os.path.expanduser("~")

In [ ]:
datadirs=glob(os.path.join(HOME,'qregress/./5qubit/*/M-M-CZ_HWE-CNOT/M-M-CZ_HWE-CNOT_results.json'))+[os.path.join(HOME,'qregress/BSE-calc-test/5qubit/M-M-CZ_HWE-CNOT/M-M-CZ_HWE-CNOT_results.json')]

In [ ]:
dfs=[]
for i in datadirs:
    with open(i,'r') as g:
        data = json.load(g)
    df = pd.DataFrame.from_dict(data)
    name = os.path.basename(os.path.dirname(os.path.dirname(i)))
    if "RUD"  in name:
        df['RUD']=int(name.replace("RUD",""))
        df['AL']=1
        # df['RUD'] = 
    elif "AL" in name:
        df['RUD']=1
        df['AL']=name.replace("AL","")
    else:
        df['RUD']=1
        df['AL']=1
    dfs.append(df)
    
stacked = pd.concat(dfs)

In [ ]:
stacked

In [ ]:


# device='qiskit.aer'
# device_backend = FakeCairoV2()
# backend = AerSimulator.from_backend(device_backend)
# device = qml.device(device, wires=num_qubits, backend=backend, shots=shots)


device='qulacs.simulator'
num_qubits=5
device = qml.device(device, wires=num_qubits)


In [ ]:
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(num_qubits))
        variational(params, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [ ]:
def get_depth(enc,var,rud,features,parameters):
    circuit_gates=0
    circuit_depth=0
    for i in range(rud):
        params = initial_parameters.reshape(rud,-1)[i]
        
        encoder_specs=qml.specs(qml.QNode(encoder,device,expansion_strategy='device'))(features, wires=range(num_qubits))['resources']
        var_specs=qml.specs(qml.QNode(variational,device,expansion_strategy='device'))(params, wires=range(num_qubits))['resources']

        circuit_gates+=var_specs.num_gates + encoder_specs.num_gates
        circuit_depth+=var_specs.depth + encoder_specs.depth
    return circuit_gates,circuit_depth

In [ ]:


# draw_encoder = qml.draw_mpl(qml.QNode(ENCODER_LIST['IQP'],device),level=3)
# draw_encoder(np.random.rand(num_qubits), range(num_qubits))

In [ ]:
results={}
flat_results=[]
for d in [3, 5]:
    results[d]={}
    for a in ['AL','RUD']:
        results[d][a]={}
        if a=='AL':
            LAYERS=d
            re_upload_depth=1
        else:
            LAYERS=1
            re_upload_depth=d
            
        for p in ["M-M-CZ_HWE-CNOT"]:
            results[d][a][p]={}
            depth_path = os.path.join("./5qubit/",''.join((str(d),a)),p)
            if os.path.exists(depth_path):
                with open(os.path.join(depth_path,f'{p}_results.json'),'r') as f:
                    results_dict=json.load(f)
                
                
            ENCODER, ANSATZ = p.split('_')
            # print(ENCODER, ANSATZ)
            encoder = ENCODER_LIST[ENCODER]
            variational=ANSATZ_LIST[ANSATZ]
            variational.layers = LAYERS
            variational.set_wires(range(num_qubits))
     

            num_params = variational.num_params * re_upload_depth
            generator = np.random.default_rng(12958234)
            initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
            feat = generator.uniform(-1, 1, num_qubits)
            
            qnode = qml.QNode(circuit, device)        
            circuit_specs=qml.specs(qnode)(feat,initial_parameters)['resources']

            circuit_gates,circuit_depth= get_depth(encoder,variational,re_upload_depth,feat,initial_parameters)
            # print(circuit_specs.num_gates,circuit_specs.depth)
            # print(circuit_gates,circuit_depth)
            
            # print(f"{p} AL_{LAYERS} RUD_{re_upload_depth}\n",circuit_specs)
            # qml.draw_mpl(qnode,level=1)(feat,initial_parameters)
            # plt.show()
            # print(variational.layers)
            # print()
            with open(os.path.join(depth_path,'model_log.csv'),'r') as f:
                # Create df
                timedf = pd.DataFrame(pd.to_datetime([i.split(',')[0] for i in f.readlines() if '[' in i], format='%a %b %d %H:%M:%S %Y'),columns=['Time'])
                # Get time between iterations and get the mean time
                timedf['Iter time']=[i.total_seconds() for i in timedf['Time'].diff()]
                
            print(d,a,p,results_dict['MAE_train'],results_dict['MAE_test'])
            results[d][a][p]={'R2_train':results_dict['R2_train'],'R2_test':results_dict['R2_test'],'num_gates':circuit_gates,'depth':circuit_depth,'mean_iter':timedf['Iter time'].mean()}
            if a=='AL':
                flat_results.append((d,1,p,results_dict['R2_train'],results_dict['R2_test'],circuit_gates,circuit_depth,timedf['Iter time'].mean()))
            else:
                flat_results.append((1,d,p,results_dict['R2_train'],results_dict['R2_test'],circuit_gates,circuit_depth,timedf['Iter time'].mean()))
            

In [ ]:
for p in ["M-M-CZ_HWE-CNOT"]:
    with open(f'../BSE-calc-test/5qubit/{p}/{p}_results.json','r') as f:
        og_results=json.load(f)
        
    ENCODER, ANSATZ = p.split('_')
    # print(ENCODER, ANSATZ)
    encoder = ENCODER_LIST[ENCODER]
    variational=ANSATZ_LIST[ANSATZ]

    LAYERS=1
    re_upload_depth=1
    variational.layers = LAYERS
    variational.set_wires(range(num_qubits))
    

    num_params = variational.num_params * re_upload_depth
    generator = np.random.default_rng(12958234)
    initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
    feat = generator.uniform(-1, 1, num_qubits)
    
    qnode = qml.QNode(circuit, device)   

    # print(f"{p} AL_{LAYERS} RUD_{re_upload_depth}\n",circuit_specs)
    # qml.draw_mpl(qnode,level=1)(feat,initial_parameters)
    # plt.show()
    
    circuit_specs=qml.specs(qnode)(feat,initial_parameters)['resources']

    circuit_gates,circuit_depth= get_depth(encoder,variational,1,feat,initial_parameters)
    
    with open(os.path.join(f"../BSE-calc-test/5qubit/{p}",'model_log.csv'),'r') as f:
        # Create df
        timedf = pd.DataFrame(pd.to_datetime([i.split(',')[0] for i in f.readlines() if '[' in i], format='%a %b %d %H:%M:%S %Y'),columns=['Time'])
        # Get time between iterations and get the mean time
        timedf['Iter time']=[i.total_seconds() for i in timedf['Time'].diff()]
    print(p,og_results['MAE_train'],og_results['MAE_test'])
    flat_results.append((1,1,p,og_results['R2_train'],og_results['R2_test'],circuit_gates,circuit_depth,timedf['Iter time'].mean()))

In [ ]:
results

In [ ]:
df = pd.DataFrame(flat_results,columns=['AL','RUD','pairs','R2 Train','R2 Test','num_gates','depth','mean_iter'])

In [ ]:
df.sort_values(by=['R2 Train','R2 Test'])

In [ ]:
trash=pd.read_csv('./5qubit/3AL/M-M-CZ_HWE-CNOT/M-M-CZ_HWE-CNOT_predicted_values.csv',index_col=0)

In [ ]:
trash

In [ ]:
plt.scatter([float(i.strip('[]')) for i in trash['Predicted']],[float(i.strip('[]')) for i in trash['Reference']])
plt.plot([float(i.strip('[]')) for i in trash['Reference']],[float(i.strip('[]')) for i in trash['Reference']])

In [ ]:
df['diff']=(df['R2 Train']-df['R2 Test']).abs()

In [ ]:
base = df[(df["AL"]==1)&(df["RUD"]==1)]['R2 Train']

In [ ]:
df[(df["AL"]!=1)|(df["RUD"]!=1)]

In [ ]:
df.sort_values(by=['R2 Train','R2 Test'])

In [ ]:
100*((df[(df["AL"]!=1)|(df["RUD"]!=1)]['R2 Train'] - base.values)/base.values)

In [ ]:
df['AL_RUD']=[f"({AL},{RUD})" for AL,RUD in df[['AL','RUD']].values]

In [ ]:
df.rename(columns={'R2 Train':'Train','R2 Test':'Test'}).melt(id_vars=['AL_RUD'], value_vars=['Train','Test']).sort_values(by='AL_RUD')

In [ ]:
ax = sns.barplot(data=df.rename(columns={'R2 Train':'Train','R2 Test':'Test'}).melt(id_vars=['AL_RUD'], value_vars=['Train','Test']).sort_values(by='AL_RUD'),x='variable',y='value',hue='AL_RUD',palette=sns.color_palette('Paired',5))
for bar in ax.patches:
    height = bar.get_height()
    if height!=0:
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            height,
            f'{height:.2f}',  # Format the label as desired
            ha='center',
            va='bottom',

        )

plt.ylabel('R$^{2}$')
plt.xlabel('')
plt.title('BSE5: M-M-CZ_HWE-CNOT')
plt.legend(bbox_to_anchor=(1.25, 1.0),title='(AL,RUD)')
plt.ylim(0,0.20)
plt.tight_layout()
plt.savefig(os.path.join(HOME,'qregress/images/BSE/fivequbit/BSE5_RUDAL.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
melteddf = df.rename(columns={'R2 Train':'Train','R2 Test':'Test'}).melt(id_vars=['AL','RUD'], value_vars=['Train','Test'])

fig, (ax1,ax2) = plt.subplots(1,2,sharey=True,figsize=(10,5))
sns.lineplot(melteddf[melteddf['RUD']==1],x='AL',y='value',hue='variable',ax=ax1,palette=sns.color_palette('Paired',2),marker='o', markeredgecolor='black')
sns.lineplot(melteddf[melteddf['AL']==1],x='RUD',y='value',hue='variable',ax=ax2,palette=sns.color_palette('Paired',2),marker='o', markeredgecolor='black')
ax1.set_ylim(0,1)
ax1.set_ylabel("R$^{2}$")
ax1.set_xlim(0.75,5.25)
ax2.set_xlim(0.75,5.25)
ax1.set_xlabel("Ansatz Layers (AL)")
ax2.set_xlabel("Re-upload Depth (RUD)")
# Remove legends from individual plots
ax1.get_legend().remove()
ax2.get_legend().remove()

# Global legend
fig.suptitle('M-M-CZ_HWE-CNOT')
plt.legend(loc="upper right", bbox_to_anchor=(1.5, 1.0))
plt.tight_layout()
plt.savefig(os.path.join(HOME,'qregress/images/BSE/fivequbit/BSE5_RUDAL_lineplot.png'),dpi=300,bbox_inches='tight')
plt.show()